In [4297]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4298]:
template = 'https://www.jobs.bg/front_job_search.php?subm=1&keywords%5B%5D={}'
#template = 'https://www.jobs.bg/front_job_search.php?subm=1&categories%5B0%5D=56&keywords%5B0%5D={}'


In [4299]:
def get_url(position):
    template = 'https://www.jobs.bg/front_job_search.php?subm=1&keywords%5B%5D={}'
    #template = 'https://www.jobs.bg/front_job_search.php?subm=1&categories%5B0%5D=56&keywords%5B0%5D={}'
    url = template.format(position)
    return url

In [4300]:
url = get_url("data scientist")

# Extract raw html

In [4301]:
response = requests.get(url)

In [4302]:
response

<Response [200]>

In [4303]:
response.reason

'OK'

In [4304]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4305]:
cards = soup.find_all('li', {'class': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20,21, 22]})

In [4306]:
len(cards)

20

# Prototype the model with a single record

In [4307]:
card = cards[0]

In [4308]:
atag = card.div.a

In [4309]:
job_title = atag.get('title')

In [4310]:
job_url = atag.get('href')

In [4311]:
company_name = card.find('div', {'class':'secondary-text'}).text.strip()

In [4312]:
job_location = card.find('div', "card-info card__subtitle").text.strip()

In [4313]:
#card.find('div',{'class':'card-tags pt-8'})

In [4314]:
date_posted = card.find('div','mdc-layout-grid__cell mdc-layout-grid__cell--span-12 flex').text.strip()[0:8]

In [4315]:
today = datetime.today().strftime('%Y-%m-%d')

In [4316]:
try:
    job_salary = card.find('div', 'card-info card__subtitle').text.strip()[26:49]
except AttributeError:
    job_salary = ''

# Generalize the model with a function

In [4317]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [4318]:
records[0:19]

[('DATA SCIENTIST',
  'Schwarz Global Services Bulgaria EOOD',
  'София',
  '28.11.22',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6604780'),
 ('Customer Analytics Data Scientist Senior Associate',
  'PricewaterhouseCoopers Bulgaria Ltd',
  'София',
  '25.11.22',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6602732'),
 ('Regulatory Senior Risk Data Scientist',
  'UniCredit Consumer Financing EAD',
  'София',
  '16.11.22',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6589852'),
 ('Data Business Analyst/ Designer',
  'UNIQA SOFTWARE - SERVICE BULGARIA Ltd.',
  'Пловд',
  'вчера\n  ',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6622968'),
 ('Data Engineer',
  'Teva / Тева Фарма ЕАД',
  'София',
  '09.12.22',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6621266'),
 ('Senior Data Engineer',
  'Accenture Bulgaria EOOD',
  'wifi ',
  '09.12.22',
  '2022-12-13',
  None,
  'https://www.jobs.bg/job/6619952'),
 ('Senior Data Engineer',
  'Accenture Bulgaria EOOD',

# Getting the next page

In [4319]:
#soup.find('ul', 'page-2')

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('li', {'class': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20]})

for card in cards:
    record = get_record(card)
    records.append(record)

In [4320]:
with open('job_data.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'CompanyName','JobLocation','PostDate','ExtractDate','JobSalary','JobUrl'])
        writer.writerows(records)

# Putting it all together

In [4321]:
# run main function
#main("Analyst")

In [4322]:
df = pd.read_csv('job_data.csv')

In [4323]:
df = df.loc[(df['JobLocation'] == 'София') | (df['JobLocation'] == 'wifi ')]
#df = datetime.date_posted().strftime('%Y-%m-%d')
#df[df['JobLocation'].str.contains('София', na=False)]
#df = df[df['JobTitle'].str.contains('Data Analyst')]
#df['PostDate'] = pd.to_datetime(df['PostDate'])


In [4324]:
df

,JobTitle,CompanyName,JobLocation,PostDate,ExtractDate,JobSalary,JobUrl
0,DATA SCIENTIST,Schwarz Global Services Bulgaria EOOD,София,28.11.22,2022-12-13,NaN,https://www.jobs.bg/job/6604780
1,Customer Analytics Data Scientist Senior Assoc...,PricewaterhouseCoopers Bulgaria Ltd,София,25.11.22,2022-12-13,NaN,https://www.jobs.bg/job/6602732
2,Regulatory Senior Risk Data Scientist,UniCredit Consumer Financing EAD,София,16.11.22,2022-12-13,NaN,https://www.jobs.bg/job/6589852
4,Data Engineer,Teva / Тева Фарма ЕАД,София,09.12.22,2022-12-13,NaN,https://www.jobs.bg/job/6621266
5,Senior Data Engineer,Accenture Bulgaria EOOD,wifi,09.12.22,2022-12-13,NaN,https://www.jobs.bg/job/6619952
6,Senior Data Engineer,Accenture Bulgaria EOOD,София,09.12.22,2022-12-13,NaN,https://www.jobs.bg/job/6619936
8,Safety Scientist,HCL Technologies / C3i Europe Ltd,София,08.12.22,2022-12-13,NaN,https://www.jobs.bg/job/6618298
9,Data Engineer,KBC Global Services BG/ Digital Development Ce...,София,17.11.22,2022-12-13,NaN,https://www.jobs.bg/job/6591725
11,Java Developer,АТОС БЪЛГАРИЯ КОМПЕТЪНСИ ЦЕНТЪР ЕООД,София,днес\n,2022-12-13,NaN,https://www.jobs.bg/job/6625051
12,DevOps ENGINEER,АТОС БЪЛГАРИЯ КОМПЕТЪНСИ ЦЕНТЪР ЕООД,София,днес\n,2022-12-13,NaN,https://www.jobs.bg/job/6624837
